In [1]:
from transformers import pipeline
import torch
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from folium.plugins import HeatMap
from itertools import permutations
import numpy as np

c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipe = pipeline("token-classification", model="akdeniz27/bert-base-turkish-cased-ner",device=0)

c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Example

In [3]:
text = "arkadaşımıza ulaşamıyoruz kahramanmaraş elbistan pınarbaşı mahallesi pınarbaşı caddesi cemre yapıcı kahramanmaraş deprem"

result = pipe(text)

print("Token ve Etiketler:")

location = ""
address_keywords = ["mahallesi", "caddesi", "sokak", "bulvarı", "köyü", "yolu", "mevkii", "mah.", "cad.", "sok.", "bul.", "köy.", "yol.", "mek.","apartman","apt"]

loc_tokens = []
for entity in result:
    token = entity['word']
    label = entity['entity']
    print(f"Token: {token}, Etiket: {label}")

    if "LOC" in label:  
        if token.startswith("##"):
            loc_tokens[-1] += token.lstrip("##")  
        else:
            loc_tokens.append(token)

words = text.split()  
for i, word in enumerate(words):
    if any(loc in word for loc in loc_tokens):
        location += word + " " 
    if any(keyword in word.lower() for keyword in address_keywords):
            location += word + " " 
loc_tokens
print("\nLocation:")
print(location.strip())

Token ve Etiketler:
Token: kahraman, Etiket: B-LOC
Token: ##maraş, Etiket: B-LOC
Token: elbis, Etiket: B-LOC
Token: ##tan, Etiket: B-LOC
Token: p, Etiket: B-LOC
Token: ##ınar, Etiket: B-LOC
Token: ##başı, Etiket: B-LOC
Token: p, Etiket: B-LOC
Token: ##ınar, Etiket: B-LOC
Token: ##başı, Etiket: B-LOC
Token: cem, Etiket: B-PER
Token: ##re, Etiket: B-PER
Token: kahraman, Etiket: B-LOC
Token: ##maraş, Etiket: B-LOC

Location:
kahramanmaraş elbistan pınarbaşı mahallesi pınarbaşı caddesi kahramanmaraş


c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
text = "çocuk enkazın altında ai̇leşi̇ i̇ki̇ gündür acil müdahale edi̇lmesi̇ni̇ bekliyor adres akasya mahallesi şükrü balcı caddesi sara apartmanı hatay antakya iletişim deprem sondakikadeprem acil acildeprem"

result = pipe(text)

print("Token ve Etiketler:")

location = ""

loc_tokens = []
for entity in result:
    token = entity['word']
    label = entity['entity']
    print(f"Token: {token}, Etiket: {label}")

    # LOC etiketli tokenları topla
    if 'LOC' in label:  
            if not token.startswith('##'):  
                if location:  
                    location += " " 
                location += token
            else:
                location += token.lstrip('##')  


print("\nLocation:")
print(location.strip())


Token ve Etiketler:
Token: ak, Etiket: B-LOC
Token: ##asy, Etiket: B-LOC
Token: ##a, Etiket: B-LOC
Token: bal, Etiket: I-PER
Token: ##cı, Etiket: I-PER
Token: sar, Etiket: B-LOC
Token: ##a, Etiket: B-LOC
Token: hata, Etiket: B-LOC
Token: ##y, Etiket: B-LOC
Token: ant, Etiket: B-LOC
Token: ##ak, Etiket: B-LOC
Token: ##ya, Etiket: B-LOC

Location:
akasya sara hatay antakya


In [5]:
geolocator = Nominatim(user_agent="geoapi")

# Adres olarak tam metni belirtiyoruz
address = "hatay esenlik mahallesi"

# Adresi enlem ve boylama çevir
location = geolocator.geocode(address)

if location:
    print(f"Adres: {address}")
    print(f"Enlem: {location.latitude}, Boylam: {location.longitude}")
else:
    print(f"Adres bulunamadı: {address}")

Adres: hatay esenlik mahallesi
Enlem: 36.2059571, Boylam: 36.1478466


# Get Location 

In [60]:
df = pd.read_csv("../results/TweetAnalyzeResult.csv")
df.head()

,timestamp,username,handle,tweet_date,content,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words
0,2024-11-03 19:38:25,Orhan Şevik,orhan_sevik,"Feb 8, 2023",@haluklevent\n @oguzhanugur\n hepinizden alla...,\n \n hepinizden allah razı olsun iyiki varsı...,hepinizden allah razı olsun iyiki varsını...,hepinizden allah razı olsun iyiki varsınız deprem,hepinizden allah razı olsun iyi ki varsınız de...,"['hepinizden', 'allah', 'razı', 'olsun', 'iyi'...","['hepinizden', 'razı', 'varsınız', 'deprem']","['hep', 'razı', 'var', 'deprem']","['hep', 'razı', 'var', 'deprem']"
1,2024-11-03 19:38:25,nursena,nurssxx_,"Feb 8, 2023",allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver deprem,allahım nolur onlara dayanma gücü ver deprem,allahım ne olur onlara dayanma gücü ver deprem,"['allahım', 'ne', 'olur', 'onlara', 'dayanma',...","['onlara', 'dayanma', 'gücü', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']"
2,2024-11-03 19:38:25,Elif ŞEKER,55ellllllif,"Feb 8, 2023",replying to \n@haluklevent\n @danlabilic\n and...,replying to \n\n \n and \n_harun\nadıyamanda a...,replying to and harun adıyamanda ali taş...,harun adıyamanda ali taşı mahallesi sokak hic...,harun adıyamanda ali taşı mahallesi sokak hicr...,"['harun', 'adıyamanda', 'ali', 'taşı', 'mahall...","['harun', 'adıyamanda', 'ali', 'taşı', 'mahall...","['haru', 'adıyaman', 'ali', 'taşı', 'mahalle',...","['adıyaman', 'ali', 'taşı', 'mahalle', 'sokak'..."
3,2024-11-03 19:38:25,Yusuf,yusufaltuns,"Feb 8, 2023",tuğba södekoğlu kovulsun \n@showtv\n #deprem,tuğba södekoğlu kovulsun \n\n #deprem,tuğba södekoğlu kovulsun deprem,tuğba södekoğlu kovulsun deprem,tuğba sodekoğlu kovulsun deprem,"['tuğba', 'sodekoğlu', 'kovulsun', 'deprem']","['tuğba', 'sodekoğlu', 'kovulsun', 'deprem']","['tuğba', 'sodekoğlu', 'kov', 'deprem']",['deprem']
4,2024-11-03 19:38:25,𝐎̈𝐳𝐠𝐮̈𝐫 𝐑𝐚𝐧,dryghtn2,"Feb 8, 2023",arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşımıza ulaşamıyoruz kahramanmaraş elbist...,"['arkadaşımıza', 'ulaşamıyoruz', 'kahramanmara...","['arkadaşımıza', 'ulaşamıyoruz', 'kahramanmara...","['arkadaş', 'ulaş', 'kahramanmaraş', 'elbistan...","['arkadaş', 'ulaş', 'kahramanmaraş', 'elbistan..."


In [6]:
def capitalize(text):
    words = text.split()
    capitalized_words = [word.capitalize() for word in words]
    capitalized_text = ' '.join(capitalized_words)
    return capitalized_text

In [7]:
file_prefix = "../ParcaliSonuc/df"
file_suffix = "_lemmatizedfix.csv"
output_prefix = "../ParcaliSonuc/df"
output_suffix = "_capitalize.csv"

for i in range(1, 21):  
    input_file = f"{file_prefix}{i}{file_suffix}"
    output_file = f"{output_prefix}{i}{output_suffix}"
    try:
        df = pd.read_csv(input_file)
        df["content_wo_tokenize_capitalize"] = df["content_wo_normalize"].apply(capitalize)
        
        df.to_csv(output_file, index=False)
        print(f"{output_file} başarıyla kaydedildi.")
    except FileNotFoundError:
        print(f"{input_file} bulunamadı.")
    except Exception as e:
        print(f"{input_file} işlenirken bir hata oluştu: {e}")

../ParcaliSonuc/df1_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df2_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df3_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df4_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df5_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df6_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df7_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df8_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df9_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df10_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df11_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df12_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df13_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df14_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df15_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df16_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df17_capitalize.csv başarıyla kaydedildi.
../ParcaliSonuc/df18_ca

In [ ]:
df['content_wo_tokenize_capitalize'] = df['content_wo_normalize'].apply(capitalize)

In [8]:
def get_location_from_text(text):
    if not text.strip():
        return None 
    result = pipe(text) 
    location = "" 
    
    for entity in result:
        token = entity['word']
        label = entity['entity']

        if 'LOC' in label:  
            if not token.startswith('##'):  
                if location:  
                    location += " " 
                location += token
            else:
                location += token.lstrip('##')  
                
    return location.strip() if location else None

In [12]:
file_prefix = "../ParcaliSonuc/df"
file_suffix = "_capitalize.csv"
output_prefix = "../ParcaliSonuc/df"
output_suffix = "_location.csv"

for i in range(1, 21):  
    input_file = f"{file_prefix}{i}{file_suffix}"
    output_file = f"{output_prefix}{i}{output_suffix}"
    try:
        df = pd.read_csv(input_file)
        df["location"] = df["content_wo_tokenize_capitalize"].apply(get_location_from_text)
        
        df.to_csv(output_file, index=False)
        print(f"{output_file} başarıyla kaydedildi.")
    except FileNotFoundError:
        print(f"{input_file} bulunamadı.")
    except Exception as e:
        print(f"{input_file} işlenirken bir hata oluştu: {e}")

../ParcaliSonuc/df1_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df2_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df3_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df4_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df5_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df6_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df7_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df8_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df9_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df10_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df11_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df12_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df13_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df14_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df15_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df16_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df17_location.csv başarıyla kaydedildi.
../ParcaliSonuc/df18_location.csv başarıyla kaydedildi.
.

In [ ]:
df['location'] = df['content_wo_tokenize_capitalize'].apply(get_location_from_text)

In [13]:
def remove_duplicates(text):
    if not text:
        return text
    words = text.split()
    unique_words = list(set(words))
    unique_words.sort(key=lambda x: words.index(x))  
    return ' '.join(unique_words)

In [33]:
file_prefix = "../ParcaliSonuc/df"
file_suffix = "_location.csv"
output_prefix = "../ParcaliSonuc/df"
output_suffix = "_locationunique.csv"

for i in range(1, 21):  
    input_file = f"{file_prefix}{i}{file_suffix}"
    output_file = f"{output_prefix}{i}{output_suffix}"
    try:
        df = pd.read_csv(input_file)

        df['location']=df['location'].astype(str)
        df["location_unique"] = df["location"].apply(remove_duplicates)
        
        df.to_csv(output_file, index=False)
        print(f"{output_file} başarıyla kaydedildi.")
    except FileNotFoundError:
        print(f"{input_file} bulunamadı.")
    except Exception as e:
        print(f"{input_file} işlenirken bir hata oluştu: {e}")

../ParcaliSonuc/df1_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df2_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df3_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df4_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df5_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df6_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df7_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df8_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df9_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df10_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df11_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df12_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df13_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df14_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df15_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/df16_locationunique.csv başarıyla kaydedildi.
../ParcaliSonuc/d

In [ ]:
df['location_unique'] = df['location'].apply(remove_duplicates)

# Special Address

In [ ]:
df2 = pd.read_csv("../ParcaliSonuc/df_locationunique.csv")
df2

,Date,Tweet,lower,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words,content_wo_tokenize_capitalize,location,location_unique
0,08.02.2023,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver deprem,allahım nolur onlara dayanma gücü ver deprem,allahım ne olur onlara dayanma gücü ver deprem,"['allahım', 'ne', 'olur', 'onlara', 'dayanma',...","['onlara', 'dayanma', 'gücü', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']",Allahım Ne Olur Onlara Dayanma Gücü Ver Deprem,NaN,NaN
1,08.02.2023,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,şu dolandırıcı ahlaksızları yakalayın ve içeri...,şu dolandırıcı ahlaksızları yakalayın ve içeri...,"['şu', 'dolandırıcı', 'ahlaksızları', 'yakalay...","['dolandırıcı', 'ahlaksızları', 'yakalayın', '...","['dolandır', 'ahlak', 'yakala', 'içeri', 'dışa...","['dolandır', 'ahlak', 'yakala', 'içeri', 'dışa...",Şu Dolandırıcı Ahlaksızları Yakalayın Ve Içeri...,NaN,NaN
2,08.02.2023,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,"['bostanbaşı', 'mahallesi', 'özpolatlar', 'sit...","['bostanbaşı', 'mahallesi', 'özpolatlar', 'sit...","['bostanbaşı', 'mahalle', 'özpolat', 'site', '...","['bostanbaşı', 'mahalle', 'özpolat', 'site', '...",Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...,Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...,Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...
3,08.02.2023,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım ancak...,hiç bir iş hakkında bunu yarın yapacağım ancak...,"['hiç', 'bir', 'iş', 'hakkında', 'bunu', 'yarı...","['hakkında', 'yarın', 'yapacağım', 'dilerse', ...","['hakkında', 'yarın', 'yap', 'dil', 'yap', 'un...","['hakkında', 'yarın', 'yap', 'dil', 'yap', 'un...",Hiç Bir Iş Hakkında Bunu Yarın Yapacağım Ancak...,NaN,NaN
4,07.02.2023,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,alevler mahallesi meltem sokak güler apartmanı...,"['alevler', 'mahallesi', 'meltem', 'sokak', 'g...","['alevler', 'mahallesi', 'meltem', 'sokak', 'g...","['alevle', 'mahalle', 'meltem', 'sokak', 'gül'...","['alevle', 'mahalle', 'meltem', 'sokak', 'gül'...",Alevler Mahallesi Meltem Sokak Güler Apartmanı...,Alevler Mahallesi Meltem Sokak Güler Apartmanı...,Alevler Mahallesi Meltem Sokak Güler Apartmanı...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42254,10.02.2023,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,ses varbuğra türkkan yenişehir mahallesi trabz...,ses varbuğra türkkan yenişehir mahallesi trabz...,ses varbuğra türkkan yenişehir mahallesi trabz...,"['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...","['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...","['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...","['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...",Ses Varbuğra Türkkan Yenişehir Mahallesi Trabz...,Yenişehir Mahallesi Trabzon Bulvarı Serçin Apa...,Yenişehir Mahallesi Trabzon Bulvarı 

In [64]:
df['tokenized_location'] = df['location_unique'].apply(lambda x: x.split() if pd.notnull(x) else [])

In [34]:
file_prefix = "../ParcaliSonuc/df"
file_suffix = "_locationunique.csv"
output_prefix = "../ParcaliSonuc/df"
output_suffix = "_tokenizedlocation.csv"

for i in range(1, 21):  
    input_file = f"{file_prefix}{i}{file_suffix}"
    output_file = f"{output_prefix}{i}{output_suffix}"
    try:
        df = pd.read_csv(input_file)
        df["tokenized_location"] = df["location_unique"].apply(lambda x: x.split() if pd.notnull(x) else [])
        
        df.to_csv(output_file, index=False)
        print(f"{output_file} başarıyla kaydedildi.")
    except FileNotFoundError:
        print(f"{input_file} bulunamadı.")
    except Exception as e:
        print(f"{input_file} işlenirken bir hata oluştu: {e}")

../ParcaliSonuc/df1_tokenizedlocation.csv başarıyla kaydedildi.


Exception ignored in: <generator object BatchSampler.__iter__ at 0x000002221E002F20>
Traceback (most recent call last):
  File "c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\torch\utils\data\sampler.py", line 291, in __iter__
    yield batch
  File "_pydevd_bundle/pydevd_cython.pyx", line 1457, in _pydevd_bundle.pydevd_cython.SafeCallWrapper.__call__
  File "_pydevd_bundle/pydevd_cython.pyx", line 1790, in _pydevd_bundle.pydevd_cython.ThreadTracer.__call__
  File "c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\debugpy\_vendored\pydevd\pydevd_file_utils.py", line 910, in get_abs_path_real_path_and_base_from_frame
    def get_abs_path_real_path_and_base_from_frame(frame, NORM_PATHS_AND_BASE_CONTAINER=NORM_PATHS_AND_BASE_CONTAINER):
    
KeyboardInterrupt: 


../ParcaliSonuc/df2_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df3_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df4_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df5_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df6_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df7_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df8_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df9_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df10_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df11_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df12_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df13_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df14_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df15_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df16_tokenizedlocation.csv başarıyla kaydedildi.
../ParcaliSonuc/df17_tokenizedloc

In [2]:
geolocator = Nominatim(user_agent="geoapi")

def get_coordinates(address):
    if not address or pd.isna(address):
        return None, None
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

In [66]:
get_coordinates("Hatay Arsuz gözcüler ")

(36.388756, 35.916489)

#### All combination

In [67]:
turkiye_adres = pd.read_excel("../data/ililcemahalle.xlsx")

turkiye_adres["il"] = turkiye_adres["il"].str.strip()
turkiye_adres["ilçe"] = turkiye_adres["ilçe"].str.strip()
turkiye_adres["semt_bucak_belde"] = turkiye_adres["semt_bucak_belde"].str.strip()
turkiye_adres["Mahalle"] = turkiye_adres["Mahalle"].str.strip()

iller = set(turkiye_adres["il"].str.lower().unique())
ilceler = set(turkiye_adres["ilçe"].str.lower().unique())
semtler = set(turkiye_adres["semt_bucak_belde"].str.lower().unique())
mahalleler = set(turkiye_adres["Mahalle"].str.lower().unique())

In [68]:
def contains_in_set(token, keyword_set):
    return [keyword for keyword in keyword_set if keyword in token]

def clean_address(address):
    return [word for word in address if 'hal' not in word.lower()]

def build_address_from_tokens(tokens):
    if not isinstance(tokens, list) or not tokens or len(tokens) == 1:
        return None  

    found_parts = []  
    tokens = [token.lower() for token in tokens]  

    for token in tokens:
        for dataset in [iller, ilceler, semtler, mahalleler]:
            if (matched_parts := contains_in_set(token, dataset)):
                if matched_parts[0] not in found_parts:
                    found_parts.append(matched_parts[0])
                break

    found_parts = clean_address(found_parts)
    return " ".join(found_parts) if found_parts else None

In [69]:
df["address"] = df["tokenized_location"].apply(build_address_from_tokens)

In [69]:
import ast

In [72]:
file_prefix = "../ParcaliSonuc/df"
file_suffix = "_tokenizedlocation.csv"
output_prefix = "../ParcaliSonuc/df"
output_suffix = "_address.csv"

for i in range(1, 21):  
    input_file = f"{file_prefix}{i}{file_suffix}"
    output_file = f"{output_prefix}{i}{output_suffix}"
    try:
        df = pd.read_csv(input_file)
        df["tokenized_location"] = df["tokenized_location"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        df["address"] = df["tokenized_location"].apply(build_address_from_tokens)
        
        df.to_csv(output_file, index=False)
        print(f"{output_file} başarıyla kaydedildi.")
    except FileNotFoundError:
        print(f"{input_file} bulunamadı.")
    except Exception as e:
        print(f"{input_file} işlenirken bir hata oluştu: {e}")

../ParcaliSonuc/df1_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df2_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df3_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df4_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df5_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df6_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df7_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df8_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df9_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df10_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df11_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df12_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df13_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df14_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df15_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df16_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df17_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df18_address.csv başarıyla kaydedildi.
../ParcaliSonuc/df1

In [73]:
df2 = pd.read_csv("../ParcaliSonuc/df2_address.csv")
df2

,Date,Tweet,lower,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words,content_wo_tokenize_capitalize,location,location_unique,tokenized_location,address
0,08.02.2023,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver deprem,allahım nolur onlara dayanma gücü ver deprem,allahım ne olur onlara dayanma gücü ver deprem,"['allahım', 'ne', 'olur', 'onlara', 'dayanma',...","['onlara', 'dayanma', 'gücü', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']",Allahım Ne Olur Onlara Dayanma Gücü Ver Deprem,NaN,NaN,[],NaN
1,08.02.2023,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,şu dolandırıcı ahlaksızları yakalayın ve içeri...,şu dolandırıcı ahlaksızları yakalayın ve içeri...,"['şu', 'dolandırıcı', 'ahlaksızları', 'yakalay...","['dolandırıcı', 'ahlaksızları', 'yakalayın', '...","['dolandır', 'ahlak', 'yakala', 'içeri', 'dışa...","['dolandır', 'ahlak', 'yakala', 'içeri', 'dışa...",Şu Dolandırıcı Ahlaksızları Yakalayın Ve Içeri...,NaN,NaN,[],NaN
2,08.02.2023,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,"['bostanbaşı', 'mahallesi', 'özpolatlar', 'sit...","['bostanbaşı', 'mahallesi', 'özpolatlar', 'sit...","['bostanbaşı', 'mahalle', 'özpolat', 'site', '...","['bostanbaşı', 'mahalle', 'özpolat', 'site', '...",Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...,Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...,Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...,"['Bostanbaşı', 'Mahallesi', 'Özpolatlar', 'Sit...",malatya
3,08.02.2023,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım ancak...,hiç bir iş hakkında bunu yarın yapacağım ancak...,"['hiç', 'bir', 'iş', 'hakkında', 'bunu', 'yarı...","['hakkında', 'yarın', 'yapacağım', 'dilerse', ...","['hakkında', 'yarın', 'yap', 'dil', 'yap', 'un...","['hakkında', 'yarın', 'yap', 'dil', 'yap', 'un...",Hiç Bir Iş Hakkında Bunu Yarın Yapacağım Ancak...,NaN,NaN,[],NaN
4,07.02.2023,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,alevler mahallesi meltem sokak güler apartmanı...,"['alevler', 'mahallesi', 'meltem', 'sokak', 'g...","['alevler', 'mahallesi', 'meltem', 'sokak', 'g...","['alevle', 'mahalle', 'meltem', 'sokak', 'gül'...","['alevle', 'mahalle', 'meltem', 'sokak', 'gül'...",Alevler Mahallesi Meltem Sokak Güler Apartmanı...,Alevler Mahallesi Meltem Sokak Güler Apartmanı...,Alevler Mahallesi Meltem Sokak Güler Apartmanı...,"['Alevler', 'Mahallesi', 'Meltem', 'Sokak', 'G...",meltem hatay antakya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42254,10.02.2023,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,ses varbuğra türkkan yenişehir mahallesi trabz...,ses varbuğra türkkan yenişehir mahallesi trabz...,ses varbuğra türkkan yenişehir mahallesi trabz...,"['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...","['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...","['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm..."

In [19]:
def update_coordinates(row):
        return get_coordinates(row['address'])


In [20]:
file_prefix = "../ParcaliSonuc/df"
file_suffix = "_address.csv"
output_prefix = "../ParcaliSonuc/df"
output_suffix = "_coordinatesaddress.csv"

for i in range(13, 21):  
    input_file = f"{file_prefix}{i}{file_suffix}"
    output_file = f"{output_prefix}{i}{output_suffix}"
    try:
        df = pd.read_csv(input_file)
        
        df["latitude"], df["longitude"] = zip(*df.apply(update_coordinates, axis=1))
        
        df.to_csv(output_file, index=False)
        print(f"{output_file} başarıyla kaydedildi.")
    except FileNotFoundError:
        print(f"{input_file} bulunamadı.")
    except Exception as e:
        print(f"{input_file} işlenirken bir hata oluştu: {e}")

../ParcaliSonuc/df13_address.csv işlenirken bir hata oluştu: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=atat%C3%BCrk+antakya&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022F92C99FD0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=10)'))


KeyboardInterrupt: 

In [ ]:
df["latitude"], df["longitude"] = zip(*df.apply(update_coordinates, axis=1))

In [16]:
df1 = pd.read_csv("../ParcaliSonuc/df1_coordinatesaddress.csv")
df1

,Date,Tweet,lower,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words,content_wo_tokenize_capitalize,location,location_unique,tokenized_location,address,latitude,longitude
0,08.02.2023,@haluklevent\n @oguzhanugur\n hepinizden alla...,@haluklevent\n @oguzhanugur\n hepinizden alla...,\n \n hepinizden allah razı olsun iyiki varsı...,hepinizden allah razı olsun iyiki varsını...,hepinizden allah razı olsun iyiki varsınız deprem,hepinizden allah razı olsun iyi ki varsınız de...,"['hepinizden', 'allah', 'razı', 'olsun', 'iyi'...","['hepinizden', 'razı', 'varsınız', 'deprem']","['hep', 'razı', 'var', 'deprem']","['hep', 'razı', 'var', 'deprem']",Hepinizden Allah Razı Olsun Iyi Ki Varsınız De...,NaN,NaN,[],NaN,NaN,NaN
1,08.02.2023,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezasısen nasıl bir vicdansız mahluk...,allah ın cezasısen nasıl bir vicdansız mahluka...,allahın cezasisen nasıl bir vicdansız mahlukat...,"['allahın', 'cezasisen', 'nasıl', 'bir', 'vicd...","['allahın', 'cezasisen', 'vicdansız', 'mahluka...","['allah', 'cezasisen', 'vicdan', 'mahlukat', '...","['allah', 'cezasisen', 'vicdan', 'mahlukat', '...",Allahın Cezasisen Nasıl Bir Vicdansız Mahlukat...,Maraş Hatay,Maraş Hatay,"['Maraş', 'Hatay']",hatay,NaN,NaN
2,08.02.2023,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış kucukdalya...,koskaca bir enkaz altındaymış kucukdalyan maha...,koskaca bir enkaz altındaymış küçükdalyan maha...,"['koskaca', 'bir', 'enkaz', 'altındaymış', 'kü...","['koskaca', 'enkaz', 'altındaymış', 'küçükdaly...","['koska', 'enkaz', 'alt', 'küçükdalyan', 'maha...","['koska', 'enkaz', 'alt', 'küçükdalyan', 'maha...",Koskaca Bir Enkaz Altındaymış Küçükdalyan Maha...,Küçükdalyan Mahallesi Emir Sokak Subaşı Apartmanı,Küçükdalyan Mahallesi Emir Sokak Subaşı Apartmanı,"['Küçükdalyan', 'Mahallesi', 'Emir', 'Sokak', ...",dalyan,NaN,NaN
3,08.02.2023,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,"['ülkeyi', 'yıkıp', 'gidecekler', 'dedikleri',...","['ülkeyi', 'yıkıp', 'gidecekler', 'dedikleri',...","['ülke', 'yık', 'gid', 'de', 'buy', 'yık', 'ül...","['ülke', 'yık', 'gid', 'de', 'buy', 'yık', 'ül...",Ülkeyi Yıkıp Gidecekler Dedikleri Buymuş Yıktı...,NaN,NaN,[],NaN,NaN,NaN
4,07.02.2023,önemli̇ duyuru \n@depremdairesi\n @afadbaskanl...,önemli̇ duyuru \n@depremdairesi\n @afadbaskanl...,önemli̇ duyuru \n\n \n _dernegi\n \n \n #adiya...,önemli̇ duyuru dernegi adiyaman depr...,önemli̇ duyuru dernegi adiyaman deprem hatayya...,önemli duyuru derneği adıyaman deprem hatayyar...,"['önemli', 'duyuru', 'derneği', 'adıyaman', 'd...","['önemli', 'duyuru', 'derneği', 'adıyaman', 'd...","['önem', 'duyuru', 'derneğ', 'adıyaman', 'depr...","['önem', 'duyuru', 'derneğ', 'adıyaman', 'depr...",Önemli Duyuru Derneği Adıyaman Deprem Hatayyar...,Adıyaman,Adıyaman,['Adıyaman'],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42249,10.02.2023,çok acill abii çokkk,çok acill abii çokkk,çok acill abii çokkk,çok acill abii çokkk,çok acill abii çokkk,çok acil tabii çökük,"['çok', 'acil', 'tabii', 'çökük']","['acil', 'çökük']","['acil', 'çökük']","['acil', 'çökük']",Çok Acil Tabii Çökük,NaN,NaN,[],NaN,NaN,NaN
42250,10.02.2023,"hatay || ses var, aci̇l çağriteyit saati: 2.0...","hatay || ses var, aci̇l çağriteyit saati: 2.0...","hatay || ses var, aci̇l çağriteyit saati: 2.0...",hatay ses var aci̇l çağriteyit saati şuba...,hatay ses var aci̇l çağriteyit saat

## City

In [7]:
sehirler = [
    "Adana", "Adıyaman", "Afyon", "Ağrı", "Amasya", "Ankara", "Antalya", "Artvin",
    "Aydın", "Balıkesir", "Bilecik", "Bingöl", "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale",
    "Çankırı", "Çorum", "Denizli", "Diyarbakır", "Edirne", "Elazığ", "Erzincan", "Erzurum", 
    "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane", "Hakkari", "Hatay", "Isparta", "Mersin",
    "İstanbul", "İzmir", "Kars", "Kastamonu", "Kayseri", "Kırklareli", "Kırşehir", "Kocaeli",
    "Konya", "Kütahya", "Malatya", "Manisa", "Kahramanmaraş", "Mardin", "Muğla", "Muş", 
    "Nevşehir", "Niğde", "Ordu", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", 
    "Tekirdağ", "Tokat", "Trabzon", "Tunceli", "Şanlıurfa", "Uşak", "Van", "Yozgat", 
    "Zonguldak", "Aksaray", "Bayburt", "Karaman", "Kırıkkale", "Batman", "Şırnak", 
    "Bartın", "Ardahan", "Iğdır", "Yalova", "Karabük", "Kilis", "Osmaniye", "Düzce"
]


### Find City

In [78]:
def extract_cities(tokens, city_list):
    if not tokens:  
        return []
    matches = []
    for city in city_list:
        for token in tokens:
            if re.search(r'\b' + re.escape(city) + r'\b', token, re.IGNORECASE): 
                matches.append(city)
    return list(set(matches))  

In [79]:
file_prefix = "../ParcaliSonuc/df"
file_suffix = "_tokenizedlocation.csv"
output_prefix = "../ParcaliSonuc/df"
output_suffix = "_city.csv"

for i in range(1, 21):  
    input_file = f"{file_prefix}{i}{file_suffix}"
    output_file = f"{output_prefix}{i}{output_suffix}"
    try:
        df = pd.read_csv(input_file)
        df["tokenized_location"] = df["tokenized_location"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        df["city"] = df["tokenized_location"].apply(lambda x: extract_cities(x, sehirler))
        
        df.to_csv(output_file, index=False)
        print(f"{output_file} başarıyla kaydedildi.")
    except FileNotFoundError:
        print(f"{input_file} bulunamadı.")
    except Exception as e:
        print(f"{input_file} işlenirken bir hata oluştu: {e}")

../ParcaliSonuc/df1_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df2_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df3_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df4_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df5_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df6_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df7_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df8_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df9_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df10_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df11_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df12_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df13_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df14_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df15_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df16_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df17_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df18_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df19_city.csv başarıyla kaydedildi.
../ParcaliSonuc/df20_

In [73]:
df['city'] = df['tokenized_location'].apply(lambda x: extract_cities(x, sehirler))

In [91]:
df2 = pd.read_csv("../ParcaliSonuc/df2_city.csv")
df2

,Date,Tweet,lower,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words,content_wo_tokenize_capitalize,location,location_unique,tokenized_location,city
0,08.02.2023,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver deprem,allahım nolur onlara dayanma gücü ver deprem,allahım ne olur onlara dayanma gücü ver deprem,"['allahım', 'ne', 'olur', 'onlara', 'dayanma',...","['onlara', 'dayanma', 'gücü', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']",Allahım Ne Olur Onlara Dayanma Gücü Ver Deprem,NaN,NaN,[],[]
1,08.02.2023,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,ya şu dolandırıcı ahlaksızları yakalayın ve iç...,şu dolandırıcı ahlaksızları yakalayın ve içeri...,şu dolandırıcı ahlaksızları yakalayın ve içeri...,"['şu', 'dolandırıcı', 'ahlaksızları', 'yakalay...","['dolandırıcı', 'ahlaksızları', 'yakalayın', '...","['dolandır', 'ahlak', 'yakala', 'içeri', 'dışa...","['dolandır', 'ahlak', 'yakala', 'içeri', 'dışa...",Şu Dolandırıcı Ahlaksızları Yakalayın Ve Içeri...,NaN,NaN,[],[]
2,08.02.2023,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,bostanbaşı mahallesi özpolatlar sitesi yeşilyu...,"['bostanbaşı', 'mahallesi', 'özpolatlar', 'sit...","['bostanbaşı', 'mahallesi', 'özpolatlar', 'sit...","['bostanbaşı', 'mahalle', 'özpolat', 'site', '...","['bostanbaşı', 'mahalle', 'özpolat', 'site', '...",Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...,Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...,Bostanbaşı Mahallesi Özpolatlar Sitesi Yeşilyu...,"['Bostanbaşı', 'Mahallesi', 'Özpolatlar', 'Sit...",['Malatya']
3,08.02.2023,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım deme ...,hiç bir iş hakkında bunu yarın yapacağım ancak...,hiç bir iş hakkında bunu yarın yapacağım ancak...,"['hiç', 'bir', 'iş', 'hakkında', 'bunu', 'yarı...","['hakkında', 'yarın', 'yapacağım', 'dilerse', ...","['hakkında', 'yarın', 'yap', 'dil', 'yap', 'un...","['hakkında', 'yarın', 'yap', 'dil', 'yap', 'un...",Hiç Bir Iş Hakkında Bunu Yarın Yapacağım Ancak...,NaN,NaN,[],[]
4,07.02.2023,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,akevler mahallesi meltem sokak güler apartmanı...,alevler mahallesi meltem sokak güler apartmanı...,"['alevler', 'mahallesi', 'meltem', 'sokak', 'g...","['alevler', 'mahallesi', 'meltem', 'sokak', 'g...","['alevle', 'mahalle', 'meltem', 'sokak', 'gül'...","['alevle', 'mahalle', 'meltem', 'sokak', 'gül'...",Alevler Mahallesi Meltem Sokak Güler Apartmanı...,Alevler Mahallesi Meltem Sokak Güler Apartmanı...,Alevler Mahallesi Meltem Sokak Güler Apartmanı...,"['Alevler', 'Mahallesi', 'Meltem', 'Sokak', 'G...",['Hatay']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42254,10.02.2023,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,⚠️ses var⚠️‼️buğra türkkan yenişehir mahallesi...,ses varbuğra türkkan yenişehir mahallesi trabz...,ses varbuğra türkkan yenişehir mahallesi trabz...,ses varbuğra türkkan yenişehir mahallesi trabz...,"['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...","['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...","['ses', 'varbuğra', 'türkkan', 'yenişehir', 'm...","['ses', 'va

### Get latitude & longitude

In [ ]:
coordinates = []

for city in sehirler:
    lat, lng = get_coordinates(city)
    coordinates.append({"city": city, "latitude": lat, "longitude": lng})

city_coordinates_df = pd.DataFrame(coordinates)
print(city_coordinates_df)

        city   latitude  longitude
0      Adana  36.986360  35.325286
1   Adıyaman  37.789360  38.314110
2      Afyon  38.685273  30.642741
3       Ağrı  39.529160  43.383564
4     Amasya  40.656945  35.772717
..       ...        ...        ...
76    Yalova  40.592898  29.077612
77   Karabük  41.111035  32.619390
78     Kilis  36.779723  37.141689
79  Osmaniye  37.251788  36.299350
80     Düzce  40.877454  31.200962

[81 rows x 3 columns]


In [10]:
import ast
file_prefix = "../ParcaliSonuc/df"
file_suffix = "_city.csv"
output_prefix = "../ParcaliSonuc/df"
output_suffix = "_coordinatecity.csv"

def update_city_coordinates(row, city_coordinates_df):
    if isinstance(row['city'], list) and len(row['city']) > 0 and pd.notna(row['city'][0]):
        city_name = row['city'][0]  
        matching_row = city_coordinates_df[city_coordinates_df['city'] == city_name]
        if not matching_row.empty:
            latitude = matching_row['latitude'].values[0]
            longitude = matching_row['longitude'].values[0]
            return latitude, longitude
    return None, None

for i in range(1, 21):  
    input_file = f"{file_prefix}{i}{file_suffix}"
    output_file = f"{output_prefix}{i}{output_suffix}"
    try:
        df = pd.read_csv(input_file)
        df["city"] = df["city"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        df["latitude_city"], df["longitude_city"] = zip(*df.apply(update_city_coordinates, axis=1, city_coordinates_df=city_coordinates_df))
        
        # Sonuçları kaydet
        df.to_csv(output_file, index=False)
        print(f"{output_file} başarıyla kaydedildi.")
    except FileNotFoundError:
        print(f"{input_file} bulunamadı.")
    except Exception as e:
        print(f"{input_file} işlenirken bir hata oluştu: {e}")

../ParcaliSonuc/df1_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df2_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df3_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df4_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df5_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df6_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df7_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df8_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df9_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df10_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df11_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df12_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df13_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df14_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df15_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/df16_coordinatecity.csv başarıyla kaydedildi.
../ParcaliSonuc/d

In [12]:
df1 = pd.read_csv("../ParcaliSonuc/df1_coordinatecity.csv")

df1

,Date,Tweet,lower,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words,content_wo_tokenize_capitalize,location,location_unique,tokenized_location,city,latitude_city,longitude_city
0,08.02.2023,@haluklevent\n @oguzhanugur\n hepinizden alla...,@haluklevent\n @oguzhanugur\n hepinizden alla...,\n \n hepinizden allah razı olsun iyiki varsı...,hepinizden allah razı olsun iyiki varsını...,hepinizden allah razı olsun iyiki varsınız deprem,hepinizden allah razı olsun iyi ki varsınız de...,"['hepinizden', 'allah', 'razı', 'olsun', 'iyi'...","['hepinizden', 'razı', 'varsınız', 'deprem']","['hep', 'razı', 'var', 'deprem']","['hep', 'razı', 'var', 'deprem']",Hepinizden Allah Razı Olsun Iyi Ki Varsınız De...,NaN,NaN,[],[],NaN,NaN
1,08.02.2023,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezasısen nasıl bir vicdansız mahluk...,allah ın cezasısen nasıl bir vicdansız mahluka...,allahın cezasisen nasıl bir vicdansız mahlukat...,"['allahın', 'cezasisen', 'nasıl', 'bir', 'vicd...","['allahın', 'cezasisen', 'vicdansız', 'mahluka...","['allah', 'cezasisen', 'vicdan', 'mahlukat', '...","['allah', 'cezasisen', 'vicdan', 'mahlukat', '...",Allahın Cezasisen Nasıl Bir Vicdansız Mahlukat...,Maraş Hatay,Maraş Hatay,"['Maraş', 'Hatay']",['Hatay'],36.345133,36.074802
2,08.02.2023,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış kucukdalya...,koskaca bir enkaz altındaymış kucukdalyan maha...,koskaca bir enkaz altındaymış küçükdalyan maha...,"['koskaca', 'bir', 'enkaz', 'altındaymış', 'kü...","['koskaca', 'enkaz', 'altındaymış', 'küçükdaly...","['koska', 'enkaz', 'alt', 'küçükdalyan', 'maha...","['koska', 'enkaz', 'alt', 'küçükdalyan', 'maha...",Koskaca Bir Enkaz Altındaymış Küçükdalyan Maha...,Küçükdalyan Mahallesi Emir Sokak Subaşı Apartmanı,Küçükdalyan Mahallesi Emir Sokak Subaşı Apartmanı,"['Küçükdalyan', 'Mahallesi', 'Emir', 'Sokak', ...",[],NaN,NaN
3,08.02.2023,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,"['ülkeyi', 'yıkıp', 'gidecekler', 'dedikleri',...","['ülkeyi', 'yıkıp', 'gidecekler', 'dedikleri',...","['ülke', 'yık', 'gid', 'de', 'buy', 'yık', 'ül...","['ülke', 'yık', 'gid', 'de', 'buy', 'yık', 'ül...",Ülkeyi Yıkıp Gidecekler Dedikleri Buymuş Yıktı...,NaN,NaN,[],[],NaN,NaN
4,07.02.2023,önemli̇ duyuru \n@depremdairesi\n @afadbaskanl...,önemli̇ duyuru \n@depremdairesi\n @afadbaskanl...,önemli̇ duyuru \n\n \n _dernegi\n \n \n #adiya...,önemli̇ duyuru dernegi adiyaman depr...,önemli̇ duyuru dernegi adiyaman deprem hatayya...,önemli duyuru derneği adıyaman deprem hatayyar...,"['önemli', 'duyuru', 'derneği', 'adıyaman', 'd...","['önemli', 'duyuru', 'derneği', 'adıyaman', 'd...","['önem', 'duyuru', 'derneğ', 'adıyaman', 'depr...","['önem', 'duyuru', 'derneğ', 'adıyaman', 'depr...",Önemli Duyuru Derneği Adıyaman Deprem Hatayyar...,Adıyaman,Adıyaman,['Adıyaman'],['Adıyaman'],37.789360,38.314110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42249,10.02.2023,çok acill abii çokkk,çok acill abii çokkk,çok acill abii çokkk,çok acill abii çokkk,çok acill abii çokkk,çok acil tabii çökük,"['çok', 'acil', 'tabii', 'çökük']","['acil', 'çökük']","['acil', 'çökük']","['acil', 'çökük']",Çok Acil Tabii Çökük,NaN,NaN,[],[],NaN,NaN
42250,10.02.2023,"hatay || ses var, aci̇l çağriteyit saati: 2.0...","hatay || ses var, aci̇l çağriteyit saati: 2.0...","hatay || ses var, aci̇l çağriteyit saati: 2.0...",hatay ses var aci̇l çağriteyit saati şuba..

In [75]:
df["latitude_city"], df["longitude_city"] = zip(*df.apply(update_city_coordinates, axis=1))

In [77]:
df.to_csv('../results/LocationResult.csv', index=False)